# ChromaDB + OpenAI — News Articles Vector Search

This notebook walks through building a **semantic search system** over a news articles dataset using:
- `polars` — fast DataFrame library for reading the CSV
- `openai` — `text-embedding-3-small` to generate vector embeddings
- `chromadb` — vector database to store, index, and query embeddings

---

### Pipeline Overview
```
CSV File (articles)
      │
      ▼
Polars DataFrame  ──► extract article text
      │
      ▼
OpenAI Embedding API  ──► text → 1536-dim vectors
      │
      ▼
ChromaDB Collection  ──► store vectors + metadata
      │
      ▼
Query  ──► embed query ──► cosine search ──► top-k results
```

---
**Dataset:** [BBC News Articles (Kaggle)](https://www.kaggle.com/datasets/hgultekin/bbcnewsarchive)  
Download `archive.zip`, extract, and upload `articles.csv` to Colab.

## Step 1 — Install Dependencies

We need three libraries that are not pre-installed in Colab:
- `polars` — faster alternative to pandas, especially for large CSVs
- `chromadb` — local vector database (no server needed)
- `openai` — OpenAI Python SDK for embedding API calls

In [ ]:
!pip install polars chromadb openai --quiet

## Step 2 — Set OpenAI API Key

Store your API key securely using **Colab Secrets** (recommended):
1. Click the 🔑 key icon in the left sidebar
2. Add a new secret named `OPENAI_API_KEY`
3. Paste your key as the value
4. Toggle **Notebook access** ON

This avoids hardcoding your key in the notebook.

In [ ]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# Quick sanity check — should print True if key is loaded
print("API key loaded:", OPENAI_API_KEY is not None and len(OPENAI_API_KEY) > 10)

## Step 3 — Load Dataset with Polars

**Why Polars instead of Pandas?**  
Polars is built in Rust and is significantly faster for reading and filtering large CSVs. The API is similar to pandas but uses lazy evaluation under the hood.

`with_row_index(offset=1)` adds an `index` column starting at 1 — we use this to generate unique document IDs for ChromaDB (`ID1`, `ID2`, ...).

**Note:** The BBC dataset uses `iso-8859-1` encoding (Latin-1), not UTF-8. Without specifying this, Polars throws a decoding error on special characters.

In [ ]:
import polars as pl

# Read CSV — iso-8859-1 encoding required for BBC dataset
# with_row_index adds a sequential ID column starting at 1
articles = pl.read_csv(
    "archive/articles.csv",
    encoding="iso-8859-1"
).with_row_index(offset=1)

print(f"Total articles: {len(articles)}")
print(f"Columns       : {articles.columns}")
print()
articles.head()

## Step 4 — Limit Dataset and Set Up Embedding Function

**Why limit to 50 articles?**  
Each article requires one OpenAI API call for embedding. At 50 articles, this costs fractions of a cent and runs in ~5 seconds. The full BBC dataset has 2000+ articles — you'd index all of them in production.

**What is `OpenAIEmbeddingFunction`?**  
This is ChromaDB's wrapper around the OpenAI embedding API. Instead of calling `openai.embeddings.create()` manually, ChromaDB calls this wrapper automatically whenever you `.add()` documents or `.query()` the collection. You register it once at collection creation and ChromaDB handles the rest.

In [ ]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

# Limit to first 50 articles for this demo
N = 50
articles = articles.head(N)
print(f"Working with {len(articles)} articles")

# Set up the OpenAI embedding function
# model: text-embedding-3-small → 1536-dimensional vectors
# This wrapper is called automatically by ChromaDB on .add() and .query()
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-3-small"
)

print("Embedding function ready: text-embedding-3-small (1536 dims)")

## Step 5 — Generate Embeddings Manually (Optional Preview)

This step calls the OpenAI API directly to generate embeddings **before** adding to ChromaDB. This is useful to:
- Inspect what a vector looks like (1536 floats)
- Understand what happens inside ChromaDB's `.add()` call
- Pre-generate embeddings if you want to cache them

**Note:** In Step 6, ChromaDB can call `openai_ef` automatically — you don't need to pre-generate embeddings unless you want to inspect or cache them.

In [ ]:
# Extract article text as a Python list
articles_list = articles["article"].to_list()

# Generate embeddings for all 50 articles via OpenAI API
# openai_ef accepts a list of strings and returns a list of vectors
print("Generating embeddings via OpenAI API...")
vectors = openai_ef(articles_list)

# Inspect the result
print(f"Number of vectors  : {len(vectors)}")
print(f"Dimensions per vector: {len(vectors[0])}")
print(f"First 5 values of vector[0]: {vectors[0][:5]}")

## Step 6 — Create ChromaDB Collection and Add Articles

**In-memory client vs Persistent client:**
```python
chromadb.Client()                        # in-memory — lost when session ends
chromadb.PersistentClient(path="./db")   # saved to disk — survives restarts
```
We use in-memory here. Step 8 shows how to switch to persistent.

**Why `get_or_create_collection`?**  
Unlike `create_collection`, this does not error if the collection already exists. Safe to re-run without resetting your Colab session.

**`hnsw:space: cosine`** — tells ChromaDB to measure similarity using cosine distance (angle between vectors). Best for text embeddings where direction matters more than magnitude.

In [ ]:
# In-memory ChromaDB client (data lives for this session only)
client = chromadb.Client()

# Create (or open) the articles collection
# embedding_function=openai_ef tells ChromaDB how to embed text for queries
articles_collection = client.get_or_create_collection(
    name="articles",
    embedding_function=openai_ef,
    metadata={"hnsw:space": "cosine"}
)

# Generate IDs — ChromaDB requires a unique string ID per document
# We use the row index from Polars: ID1, ID2, ID3, ...
ids = [f"ID{x}" for x in articles["index"].to_list()]

# Add all articles with their pre-generated embeddings
# Since we already called openai_ef above, we pass embeddings directly
# to avoid paying for a second API call
articles_collection.add(
    documents=articles_list,   # raw text (stored alongside vectors)
    ids=ids,                   # unique IDs
    embeddings=vectors         # pre-generated 1536-dim vectors
)

print(f"Collection name    : {articles_collection.name}")
print(f"Documents indexed  : {articles_collection.count()}")
print(f"Sample IDs         : {ids[:5]}")

## Step 7 — Query the Collection

**Two ways to query:**

```python
# Option A — query_texts (ChromaDB embeds the query automatically)
results = collection.query(query_texts=["your question"], n_results=3)

# Option B — query_embeddings (you embed the query manually first)
query_vec = openai_ef(["your question"])
results = collection.query(query_embeddings=query_vec, n_results=3)
```

Option B is used here (same as the video) to show the intermediate embedding step explicitly. Both produce identical results.

**What `results` contains:**
- `results["ids"]` — list of matched document IDs
- `results["documents"]` — the original article text
- `results["distances"]` — cosine distance (lower = more similar)

**Distance → Similarity:** `similarity = 1.0 - distance`

In [ ]:
# ── Query 1: Exact keyword-style query ──────────────────────────────────────
query_1 = "public transport fares by 7%"

# Embed the query using the same OpenAI model used at index time
# CRITICAL: query and documents MUST use the same embedding model
# Mixing models produces meaningless similarity scores
query_1_vec = openai_ef([query_1])

results_1 = articles_collection.query(
    query_embeddings=query_1_vec,
    n_results=3
)

print(f"Query: '{query_1}'")
print(f"{'─'*60}")
for rank, (doc_id, doc, dist) in enumerate(
    zip(results_1["ids"][0], results_1["documents"][0], results_1["distances"][0]), 1
):
    similarity = 1.0 - dist
    print(f"[Rank {rank}] {doc_id} | Similarity: {similarity:.3f}")
    print(f"  {doc[:150]}...")
    print()

In [ ]:
# ── Query 2: Natural language / semantic query ───────────────────────────────
# Notice: no exact keyword overlap with articles — pure semantic retrieval
query_2 = "What's the deal with the price of petrol?"

query_2_vec = openai_ef([query_2])

results_2 = articles_collection.query(
    query_embeddings=query_2_vec,
    n_results=2
)

print(f"Query: '{query_2}'")
print(f"{'─'*60}")
for rank, (doc_id, doc, dist) in enumerate(
    zip(results_2["ids"][0], results_2["documents"][0], results_2["distances"][0]), 1
):
    similarity = 1.0 - dist
    print(f"[Rank {rank}] {doc_id} | Similarity: {similarity:.3f}")
    print(f"  {doc[:150]}...")
    print()

In [ ]:
# ── Query 3: Try your own query ──────────────────────────────────────────────
# Change this to anything — sports, politics, economy, technology
your_query = "football premier league results"

your_vec = openai_ef([your_query])
your_results = articles_collection.query(
    query_embeddings=your_vec,
    n_results=3
)

print(f"Query: '{your_query}'")
print(f"{'─'*60}")
for rank, (doc_id, doc, dist) in enumerate(
    zip(your_results["ids"][0], your_results["documents"][0], your_results["distances"][0]), 1
):
    similarity = 1.0 - dist
    print(f"[Rank {rank}] {doc_id} | Similarity: {similarity:.3f}")
    print(f"  {doc[:150]}...")
    print()

## Step 8 — Persistent Storage

**Problem with `chromadb.Client()`:**  
The in-memory client loses all data when the Colab session ends. Every restart requires re-embedding all articles (API cost + time).

**Solution — `PersistentClient`:**  
Saves the HNSW index and all vectors to disk at the specified path. On restart, load the same client and your collection is immediately available — no re-embedding needed.

```
VectorDB/
  └── chroma.sqlite3       ← stores metadata, document text, IDs
  └── <collection-uuid>/
        └── data_level0.bin  ← the HNSW graph (the actual vectors)
```

In Colab, mount Google Drive first to make the storage permanent across sessions.

In [ ]:
import os

# ── Option A: Save to local Colab disk (lost when session ends) ──────────────
persistent_client = chromadb.PersistentClient(path="VectorDB")

# Create the collection in the persistent client
persistent_collection = persistent_client.get_or_create_collection(
    name="articles_persistent",
    embedding_function=openai_ef,
    metadata={"hnsw:space": "cosine"}
)

# Add documents (only if not already indexed)
if persistent_collection.count() == 0:
    persistent_collection.add(
        documents=articles_list,
        ids=ids,
        embeddings=vectors    # reuse pre-generated vectors — no new API call
    )
    print(f"Indexed {persistent_collection.count()} articles to disk")
else:
    print(f"Collection already has {persistent_collection.count()} articles — skipping re-indexing")

# Verify files were created on disk
print("\nFiles saved to disk:")
for root, dirs, files in os.walk("VectorDB"):
    for f in files:
        fpath = os.path.join(root, f)
        size_kb = os.path.getsize(fpath) / 1024
        print(f"  {fpath}  ({size_kb:.1f} KB)")

In [ ]:
# ── Verify persistent collection works by querying it ───────────────────────
test_query = "economic growth inflation rates"
test_vec   = openai_ef([test_query])

test_results = persistent_collection.query(
    query_embeddings=test_vec,
    n_results=2
)

print(f"Query from persistent collection: '{test_query}'")
print(f"{'─'*60}")
for rank, (doc_id, doc, dist) in enumerate(
    zip(test_results["ids"][0], test_results["documents"][0], test_results["distances"][0]), 1
):
    similarity = 1.0 - dist
    print(f"[Rank {rank}] {doc_id} | Similarity: {similarity:.3f}")
    print(f"  {doc[:150]}...")
    print()

## Summary

| Step | What Happened | Key Concept |
|---|---|---|
| Step 3 | Loaded CSV with Polars | `with_row_index` adds IDs |
| Step 4 | Set up OpenAI embedding function | Wrapper registered with ChromaDB |
| Step 5 | Generated 1536-dim vectors via API | Text → numbers representing meaning |
| Step 6 | Added articles + vectors to ChromaDB | HNSW graph index built |
| Step 7 | Queried with semantic questions | Cosine distance → similarity score |
| Step 8 | Saved to disk with PersistentClient | Survives session restarts |

---

### Key Takeaway
Query 2 (`"What's the deal with the price of petrol?"`) contains **zero exact words** that appear in any article about fuel prices — yet ChromaDB returns the correct articles. This is semantic search working — the query vector and the relevant article vectors point in the same direction in 1536-dimensional space because the embedding model learned that *petrol*, *fuel*, *oil prices*, and *price of petrol* are semantically related concepts.